In [1]:
import sys
import boto3
import os
from datetime import time, timedelta , datetime
import time as tm
import glob
from pathlib import Path
import sys
import datetime
from datetime import datetime
import pytz
import re
import pathlib
import requests
import ffmpeg
import datetime

import mysql.connector
import whisper
import numpy as np
import torch


mydb = mysql.connector.connect(
    host="workstories.cgmgwhqhbj1b.us-east-1.rds.amazonaws.com",
    user="user",
    password="password",
    database="database",
    port = "port"
)


utc=pytz.UTC
bucket_name = "workstories"
cur_path = os.getcwd()
folder_path = r'C:\Users\singh\downloaded_s3'
latest = datetime.datetime(2022, 12, 22, 0, 0,0,tzinfo = utc)
client = boto3.client(service_name = 's3',
                      aws_access_key_id= 'aws_access_key_id',
                      aws_secret_access_key='secret_access_key')

paginator = client.get_paginator('list_objects_v2')
mycursor = mydb.cursor()
while True:
    list_files = [obj for page in paginator.paginate(Bucket="workstories") for obj in page["Contents"] if obj['LastModified'] > latest]
    for file in list_files:
        url = 'https://workstories.s3.amazonaws.com/'
        item_value = file['Key']
        url = url+file['Key']
        x = requests.head(url)
        file_format = x.headers['Content-Type']
        match = re.search("/", file_format)
        res = file_format[match.end():]
        filename = os.path.join(folder_path, file['Key']+'.'+res)
        current = file['LastModified']
        
        if (current > latest and pathlib.Path(filename).suffix == '.mp4'):
            print("Downloading File: ", filename , current)
            print("\n")
            client.download_file(
                Bucket = "workstories",
                Key = file['Key'],
                Filename = str(filename))
            model = whisper.load_model("base")
            result = model.transcribe(filename)
            transcripted_text = result["text"]
            print("transcribe result = ", transcripted_text)
            
            mycursor.execute ("""UPDATE media 
            SET transcription = %s
            WHERE item=%s""", (transcripted_text, item_value))
            mydb.commit()
            os.remove(filename)
            latest = current
            
mycursor.close()
mydb.close()            



C:\Users\singh\anaconda3\envs\whisper\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\singh\anaconda3\envs\whisper\lib\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':
C:\Users\singh\anaconda3\envs\whisper\lib\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':

KeyboardInterrupt



In [ ]:
pip freeze >requirements.txt
